### Этот скрипт предназначен для парсинга страницы https://lk.msu.ru/cabinet/marks

Для автоматического парсинга предмета, оценок и зачётных единиц

**Вход скрипта:** Имя html файла (скачанной страницы с оценками). Обычно называется Оценки.html

**Выход скрипта:** Таблица с оценками в txt, удобная для вставки в excel.

**Импорты и загрузка файла**

In [ ]:
from bs4 import BeautifulSoup
import glob
import math

In [ ]:
NAME = glob.glob("*Оценки*.htm*")[0]

with open(NAME) as fp:
    soup = BeautifulSoup(fp, 'html.parser')

**Парсинг до момента списка из нужных элементов**

In [ ]:
content = soup.find("div", {"id": "w0"})

In [ ]:
content = content.find_all('table')[0].findAll('tr')[1:]

**Вытаскивание нужных данных из каждого элемента**

In [ ]:
def get_values(leader):
    if "text-success" in str(leader):
        diploma = 1
    else:
        diploma = 0
    term = leader.find_all('td')[0].get_text()
    mark = leader.find_all('td')[2].get_text()
    subject = leader.find_all('td')[1].get_text().split("Зачетных единиц:")[0]
    
    score = leader.find_all('td')[1].find('strong').get_text()
    return term, subject, mark, score, diploma

In [ ]:
res = [get_values(item) for item in content] # (term, subject, mark, score, diploma) 

**Подсчёты результатов**

In [ ]:
marks = []
for item in res:
    marks.append(item[2])
total = sum([int(float(item[3])) for item in res])

In [ ]:
print("Зачётных единиц суммарно:", total)
print("Отлично:", marks.count("отлично"))
print("Хорошо: ", marks.count("хорошо"))
print("Удовлетворительно: ", marks.count("удов."))
print("Зачтено: ", marks.count("зачтено"))

**Сохранение в .txt**

In [ ]:
with open(NAME.replace(".html", ".txt").replace(".htm", ".txt"), "w+") as f:
    for item in res:
        term, subject, mark, score, diploma = item
        total += float(score)
        f.write(f"{term}\t{subject}\t{mark}\t{score}\t{diploma}\n")
    print(f"""Result saved in '{NAME.replace(".html", ".txt").replace(".htm", ".txt")}'""")